In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess/dataset"
import os
import sys
file_path = "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess"
os.chdir(file_path)

In [ ]:
import torch
from transformers import AutoModel, AutoConfig
import os

def convert_to_transformers(paths: list, model_name: str, save_path: str):
    """
    将原生PyTorch模型权重转换为transformers库可加载的格式。

    Args:
        paths (list): 原始PyTorch模型文件路径列表。
        model_name (str): 对应的transformers模型名称，用于加载正确的配置。
                          例如: 'bert-base-uncased', 'gpt2' 等。
        save_path (str): 转换后模型保存的本地目录路径。
    """
    if not paths:
        print("模型路径列表为空，无法进行转换。")
        return

    # 1. 组合所有分片权重到一个字典
    full_state_dict = {}
    print("正在加载所有模型权重分片...")
    for i, path in enumerate(paths):
        print(f"  > 加载: {path}")
        # 如果模型有多个分片，需要将它们合并
        shard_state_dict = torch.load(path, map_location='cpu')
        full_state_dict.update(shard_state_dict)

    print("所有权重已加载。")

    # 2. 从 Hugging Face Hub 加载对应的模型配置
    # 这一步非常重要，因为 transformers 模型需要一个 config.json 文件
    # 来定义模型结构。
    try:
        config = AutoConfig.from_pretrained(model_name)
    except Exception as e:
        print(f"加载模型 {model_name} 的配置失败，请检查模型名称是否正确。错误: {e}")
        return

    # 3. 创建一个新的 transformers 模型实例
    # 这里的 AutoModel 会根据 config 自动实例化正确的模型类。
    try:
        model = AutoModel.from_config(config)
    except Exception as e:
        print(f"创建模型实例失败，请确保模型名称 {model_name} 与你的权重结构匹配。错误: {e}")
        return

    # 4. 匹配并加载权重
    print("正在将权重加载到新的 transformers 模型实例中...")
    # 这里我们假设原始模型的 state_dict 键名与 transformers 库的命名方式完全一致。
    # 如果不一致，你需要在此处编写一个映射函数来处理键名转换。

    # 例如，如果你的原始键名是 'fc1.weight'，而 transformers 是 'linear1.weight'，
    # 你需要像下面这样做：
    # mapped_state_dict = {
    #     'linear1.weight': full_state_dict['fc1.weight'],
    #     'linear1.bias': full_state_dict['fc1.bias'],
    #     ...
    # }

    # 正常情况下，直接加载即可
    model.load_state_dict(full_state_dict)

    print("权重加载成功。")

    # 5. 保存为 transformers 格式
    print(f"正在将模型保存到 {save_path}...")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    model.save_pretrained(save_path)

    print("转换完成！你可以使用以下代码加载模型：")
    print(f"```python\nfrom transformers import AutoModel\nmodel = AutoModel.from_pretrained('{save_path}')\n```")

# --- 使用示例 ---
if __name__ == '__main__':
    model_paths = ["./out/pretrain_512.pth", "./out/full_sft_512.pth", "./out/lora_512.pth", "./out/rlhf_512.pth"]
    hf_model_name = 'bert-base-uncased' # 确保这个名称和你模型的结构匹配
    output_directory = './miniformers'
    convert_to_transformers(model_paths, hf_model_name, output_directory)
